In [1]:
import pandas as pd
from functools import partial
import multiprocessing
from multiprocessing import Pool
import numpy as np

In [2]:
import time
from datetime import datetime, timedelta
from pytz import timezone


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
# thanks to https://stackoverflow.com/questions/4182603/how-to-convert-a-string-to-utf-8-in-python
def make_unicode(input):
    if type(input) != unicode:
        input =  input.decode('utf-8')
        return input
    else:
        return input

In [6]:
#chats_beta = pd.read_csv("../data/sample/chats_beta.csv")
#chats_beta = pd.read_csv("../data/train/tickers_train.csv")
chats_beta = pd.read_csv("../data/train/onchain/TRX_train.csv")


In [7]:
chats_beta.tail()

,from,to,value,timestamp
469248,11eeb1,11aad9,7.873947e+10,1525054091
469249,4481f,14ac53,1.000000e+07,1525054091
469250,bc9b7,96d1f,3.597031e+09,1525054108
469251,23cdc,95d21,9.293694e+08,1525054108
469252,f4b10,2077e,3.208266e+09,1525054108


In [8]:
#chats_beta = chats_beta.head(50)

In [9]:
chats_beta.shape

(469253, 4)

In [10]:
datetime.strptime("2018-03-01T00:00:00Z","%Y-%m-%dT%H:%M:%SZ")

datetime.datetime(2018, 3, 1, 0, 0)

create a column with string date

In [11]:
def markDataFrameByChunks(df, num, col):
    ordNums = np.array(sorted(df[col].unique(),reverse=False))
    ordRange = range(0, len(ordNums))
    chunksDF = pd.DataFrame({'colName': ordNums, 'theirNum': ordRange}, index = ordRange)
    chunksDF['chunk'] = pd.cut(chunksDF['theirNum'], num)
    return chunksDF

In [12]:
# assign chunk numbers
colname = "from"
chats_betaWithChunks = markDataFrameByChunks(chats_beta,4,colname)
tmpDF = chats_beta.merge(chats_betaWithChunks, left_on = colname, right_on = 'colName', how = 'left')
tmpDF.drop(['colName','theirNum'], axis = 1, inplace = True)
print tmpDF.shape

(469253, 5)


In [13]:
tmpDF.tail()

,from,to,value,timestamp,chunk
469248,11eeb1,11aad9,7.873947e+10,1525054091,"(-144.038, 36009.5]"
469249,4481f,14ac53,1.000000e+07,1525054091,"(36009.5, 72019.0]"
469250,bc9b7,96d1f,3.597031e+09,1525054108,"(108028.5, 144038.0]"
469251,23cdc,95d21,9.293694e+08,1525054108,"(36009.5, 72019.0]"
469252,f4b10,2077e,3.208266e+09,1525054108,"(108028.5, 144038.0]"


In [14]:
def parallel_date_unix_convert(colname, df):
    #print df.shape
    #print df[1].columns
    #lang_dict = {}
    lang_list = []
    for index, row in df[1].iterrows():
        lang_dict = {}
        lang_dict['index_orig'] = index
        try:
            date_date = datetime.fromtimestamp(row[colname])
            #date_date = datetime.strptime(row[colname],"%Y-%m-%dT%H:%M:%SZ")
            #Sprint date_date
        except:
            date_date = None
        if not(pd.isnull(date_date)):
            lang_dict['date_datetime'] = date_date
            lang_dict['date_date'] = date_date.date()
            lang_dict['date_string'] = date_date.strftime('%Y-%m-%d %H:%M:%S')
        else:
            lang_dict['date_date'] = None
            lang_dict['date_string'] = None
            lang_dict['date_datetime'] = None
        lang_list.append(lang_dict)
    return lang_list

In [15]:
def parallel_date_unix_convert_test(colname, df):
    #print df.shape
    #print df[1].columns
    #lang_dict = {}
    lang_list = []
    for index, row in df[1].iterrows():
        lang_dict = {}
        lang_dict['index_orig'] = index
        date_date = datetime.strptime(row[colname],"%Y-%m-%dT%H:%M:%SZ")
        print date_date
    return lang_list

In [16]:
%%time
#dfByChunk = dfByChunk.groupby('chunk')
dfByChunk = tmpDF.groupby('chunk')
#func = partial(parallel_date_unix_convert_test, "datetime")
func = partial(parallel_date_unix_convert, "timestamp")
if __name__ == '__main__':
    p = Pool(processes=3)
    print 'defined workers'
    result = (p.map(func, dfByChunk))
    p.close()
    try:
        p.terminate()
    except WindowsError:
        pass
    p.join()
    dfByChunk = None
    print 'closed pool'
p.terminate()

defined workers
closed pool
CPU times: user 1.35 s, sys: 207 ms, total: 1.56 s
Wall time: 32.3 s


In [17]:
tmpDF = None

In [18]:
%%time
# unpack list with outputs
unpackedDF = pd.DataFrame()
for res in result:
    #print res
    tmpDF = pd.DataFrame(res)
    unpackedDF = pd.concat([unpackedDF, tmpDF])

CPU times: user 1.04 s, sys: 32.6 ms, total: 1.07 s
Wall time: 1.01 s


In [19]:
unpackedDF.set_index('index_orig', inplace = True);

In [20]:
unpackedDF.head()

,date_date,date_datetime,date_string
index_orig,,,
1,2018-03-07,2018-03-07 00:08:13,2018-03-07 00:08:13
4,2018-03-07,2018-03-07 00:10:37,2018-03-07 00:10:37
5,2018-03-07,2018-03-07 00:10:47,2018-03-07 00:10:47
11,2018-03-07,2018-03-07 00:11:48,2018-03-07 00:11:48
13,2018-03-07,2018-03-07 00:12:18,2018-03-07 00:12:18


In [21]:
chats_beta_with_time = chats_beta.merge(unpackedDF, left_index = True, right_index = True)

In [22]:
chats_beta_with_time.head()

,from,to,value,timestamp,date_date,date_datetime,date_string
0,aca5b,52aa9,1.000000e+08,1520370474,2018-03-07,2018-03-07 00:07:54,2018-03-07 00:07:54
1,113582,8d0d5,4.995468e+09,1520370493,2018-03-07,2018-03-07 00:08:13,2018-03-07 00:08:13
2,8ef0d,104fc4,1.000000e+09,1520370524,2018-03-07,2018-03-07 00:08:44,2018-03-07 00:08:44
3,60dbe,69d9e,3.670003e+11,1520370548,2018-03-07,2018-03-07 00:09:08,2018-03-07 00:09:08
4,11416e,69d9e,3.997940e+11,1520370637,2018-03-07,2018-03-07 00:10:37,2018-03-07 00:10:37


In [23]:
chats_beta = None
unpackedDF = None

In [24]:
chats_beta_with_time.to_pickle("../data/train/onchain/onchain_train_trx_withDates.pickle")